In [12]:
import h5py
import scanpy as sc
import anndata
import loompy as lp

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import sys
import glob
import importlib

In [13]:
import sys
sys.path.insert(0, '/home/tchari/monod/src/')

In [14]:
import monod
importlib.reload(monod)
from monod import preprocess, extract_data, cme_toolbox, analysis, mminference

In [15]:
from sklearn.metrics import adjusted_rand_score, adjusted_mutual_info_score
from sklearn.cluster import KMeans
from sklearn.neighbors import kneighbors_graph

import igraph as ig
import leidenalg as la

In [16]:
# ********* Get meK_looms.tar.gz and meKMeans_data_samp_params.csv *********

samps = pd.read_csv('meKMeans_data_samp_params.csv')
samps.head()

,Dataset,Transcrip,Clusters,Cu,Lam
0,allen_b08,/home/tchari/perturbCME/notebooks/gg_200524_mo...,10,-6.777778,-1.25
1,allen_b02h01,/home/tchari/perturbCME/notebooks/gg_200524_mo...,11,-6.777778,-1.25
2,cl3,/home/tchari/perturbCME/notebooks/gg_200525_ge...,3,-6.777778,-1.25
3,cl5,/home/tchari/perturbCME/notebooks/gg_200525_ge...,5,-6.777778,-0.70
4,brca1,/home/tchari/perturbCME/notebooks/gg_200524_mo...,-1,-6.777778,-0.70


In [17]:
d_strings  = glob.glob('./hvg_objs/*.loom')
d_strings 

['./hvg_objs/allen_b08_300hvgs.loom',
 './hvg_objs/cl5_2000hvgs.loom',
 './hvg_objs/allen_b02h01_300hvgs.loom',
 './hvg_objs/cl5_300hvgs.loom',
 './hvg_objs/allen_b08_2000hvgs.loom',
 './hvg_objs/allen_b02h01_1000hvgs.loom',
 './hvg_objs/allen_b08_4000hvgs.loom',
 './hvg_objs/cl3_1000hvgs.loom',
 './hvg_objs/allen_b08_1000hvgs.loom',
 './hvg_objs/cl3_2000hvgs.loom',
 './hvg_objs/cl5_1000hvgs.loom',
 './hvg_objs/cl5_4000hvgs.loom',
 './hvg_objs/allen_b02h01_4000hvgs.loom',
 './hvg_objs/cl3_4000hvgs.loom',
 './hvg_objs/allen_b02h01_2000hvgs.loom',
 './hvg_objs/cl3_300hvgs.loom']

In [18]:
datas = pd.unique(samps.Dataset)
datas

array(['allen_b08', 'allen_b02h01', 'cl3', 'cl5', 'brca1', 'pbmc',
       'e11e13'], dtype=object)

In [19]:
attribute_names =[('unspliced','spliced'),'gene_name','barcode']

### **meK-Means**

Test with 2, correct K, K+5, K+10

In [24]:
# !rm -r /home/tchari/perturbCME/notebooks/fits/meKruns_0209

In [36]:
#For each loom in looms from ./hvg_objs

!mkdir /home/tchari/perturbCME/notebooks/fits/meKruns_0209

dir_strings = []
data_strings = []
result_strings = []

#Save method, clustering, mat_in (U,S, etc), hyper (K or res), hvgs, data, ARI, AMI
meK_res = pd.DataFrame()
meths = []
result = []
mats = []
hyper = []
hvgs = []
data = []
ari = []
ami = []

for d in d_strings:
    
    #Get tech param by keyword (datas) in loom paths
    ind = [i in d for i in datas]
    
    #Inference params for sampling
    tech_tup = [list(samps.Cu[ind])[0],list(samps.Lam[ind])[0]]
    
    #Preprocessing params for filepaths
    trans = list(samps.Transcrip[ind])[0]
    k = list(samps.Clusters[ind])[0]
    data_name = list(samps.Dataset[ind])[0]
    
    
    ds = lp.connect(d)
    num_genes = len(ds.ra[attribute_names[1]])
    num_cells = len(ds.ca[attribute_names[2]])
    true_labs = list(ds.ca['subclass_label'])
    ds.close()
    print(data_name+' with '+str(num_genes)+' HVGs')
    
    Ks = [k,k+5] #2,

    for clus in Ks:
        # ******** TRY SETTING GENE MARKERS TO ONES THAT MEET VAR AND MEAN THRESH *************
        #Run Monod preprocessing to set up files for run, leave room for genes to be removed with low U/S
        dir_string,dataset_string = monod.preprocess.construct_batch([d], \
                                                     trans, \
                                                     [data_name], \
                                                     attribute_names=attribute_names,\
                                                     batch_location='./fits/meKruns_0209', \
                                                     meta=data_name+'_K'+str(clus), \
                                                     batch_id=1, \
                                                     n_genes=100,exp_filter_threshold=None,viz=False) #num_genes-60
        dir_strings += [dir_string]
        data_strings += [dataset_string]

        #Set model and run meK-Means
        
        #Define bounds for params
        phys_lb = [-2.0, -1.8, -1.8 ] 
        phys_ub = [4.2, 2.5, 2.5] 
        samp_lb = tech_tup 
        samp_ub = tech_tup  
        gridsize = [1,1] #Already have tech params
    

        epochs = 10

        # ---------------- meK-Means Inference ---------------- 
        #Define model with bursty transcription and Poisson molecule capture/sampling
        fitmodel = monod.cme_toolbox.CMEModel('Bursty','Poisson')

        #Set up mminference parameters
        inference_parameters = monod.mminference.InferenceParameters(phys_lb,phys_ub,samp_lb,samp_ub,gridsize,\
                    dataset_string[0],fitmodel,k=clus,epochs=epochs,use_lengths = True,
                    gradient_params = {'max_iterations':5,'init_pattern':'moments','num_restarts':1})

        #Read in loom file with filtered barcodes
        search_data = monod.extract_data.extract_data(d, trans, data_name,
                    dataset_string[0], dir_string, viz=False, dataset_attr_names=attribute_names)

        #Run inference(fit_all_grid_points()) and Save result file strings
        full_result_string = inference_parameters.fit_all_grid_points(40,search_data) 

        result_strings.append(full_result_string)
        

        # ----------------  Save output ---------------- 
        #Read in results and get cluster assignments
        sr = [monod.analysis.load_search_results(i) for i in full_result_string]
        
        
        cat_assigns = np.array([None]*num_cells) #cat_assigns is empty list of length = no. cell barcodes
        for i in range(len(sr)):
            r = sr[i]
            cat_assigns[r.filt] = r.assigns #denotes which cluster (k), cells were assigned to

        print('No. clus: ',len(np.unique(cat_assigns)))
        

        meths += ['meK-Means']
        result += [cat_assigns]
        mats += ['U,S']
        hyper += [clus]
        hvgs += [num_genes]
        data += [data_name]
        ari += [adjusted_rand_score(true_labs,cat_assigns)]
        ami += [adjusted_mutual_info_score(true_labs,cat_assigns)]


meK_res['Method'] = meths
meK_res['Clustering'] = result
meK_res['Matrices'] = mats
meK_res['Hyperparam'] = hyper
meK_res['HVGs'] = hvgs
meK_res['Data'] = data
meK_res['ARI'] = ari
meK_res['AMI'] = ami
    

mkdir: cannot create directory ‘/home/tchari/perturbCME/notebooks/fits/meKruns_0209’: File exists
allen_b08 with 299 HVGs


100%|████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [00:07<00:00,  1.25it/s]


mstep self.weights:  [0.21751044 0.03273711 0.04588703 0.12264906 0.01062673 0.16432159
 0.07502246 0.08136717 0.19834032 0.05153811]
Q Function:  -919121.5686170778



100%|██████████████████████████████████████████████████████████████████████████████████████████████████████| 9/9 [00:07<00:00,  1.22it/s]


mstep self.weights:  [2.43639871e-01 8.43254838e-02 4.45307327e-02 1.07319418e-01
 1.76440338e-10 4.66575555e-02 7.13684317e-02 1.06535850e-01
 2.41612046e-01 5.40106103e-02]
Q Function:  -856697.6632118325



...

mstep self.weights:  [8.58998914e-02 7.99775784e-02 2.54194211e-10 2.54194204e-10
 1.26616969e-01 6.74705890e-01 2.54195219e-10 3.27996700e-02]
Q Function:  -546590.1764723647



100%|██████████████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:05<00:00,  1.10s/it]


mstep self.weights:  [8.57023424e-02 7.27528263e-02 2.54194204e-10 2.54194204e-10
 1.44336608e-01 6.84072447e-01 2.54194219e-10 1.31357760e-02]
Q Function:  -505878.77460666036



100%|██████████████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:05<00:00,  1.09s/it]


mstep self.weights:  [8.70303889e-02 7.31661857e-02 2.54194206e-10 2.54194204e-10
 1.52550967e-01 6.86204411e-01 2.54194380e-10 1.04804673e-03]
Q Function:  -506405.759448412

No. clus:  5


Old run below

In [25]:
# #For each loom in looms from ./hvg_objs

# !mkdir /home/tchari/perturbCME/notebooks/fits/meKruns_0209

# dir_strings = []
# data_strings = []
# result_strings = []

# #Save method, clustering, mat_in (U,S, etc), hyper (K or res), hvgs, data, ARI, AMI
# meK_res = pd.DataFrame()
# meths = []
# result = []
# mats = []
# hyper = []
# hvgs = []
# data = []
# ari = []
# ami = []

# for d in d_strings:
    
#     #Get tech param by keyword (datas) in loom paths
#     ind = [i in d for i in datas]
    
#     #Inference params for sampling
#     tech_tup = [list(samps.Cu[ind])[0],list(samps.Lam[ind])[0]]
    
#     #Preprocessing params for filepaths
#     trans = list(samps.Transcrip[ind])[0]
#     k = list(samps.Clusters[ind])[0]
#     data_name = list(samps.Dataset[ind])[0]
    
    
#     ds = lp.connect(d)
#     num_genes = len(ds.ra[attribute_names[1]])
#     num_cells = len(ds.ca[attribute_names[2]])
#     true_labs = list(ds.ca['subclass_label'])
#     ds.close()
#     print(data_name+' with '+str(num_genes)+' HVGs')
    
#     Ks = [2,k,k+5,k+10]

#     for clus in Ks:
#         # ******** TRY SETTING GENE MARKERS TO ONES THAT MEET VAR AND MEAN THRESH *************
#         #Run Monod preprocessing to set up files for run, leave room for genes to be removed with low U/S
#         dir_string,dataset_string = monod.preprocess.construct_batch([d], \
#                                                      trans, \
#                                                      [data_name], \
#                                                      attribute_names=attribute_names,\
#                                                      batch_location='./fits/meKruns_0209', \
#                                                      meta=data_name+'_K'+str(clus), \
#                                                      batch_id=1, \
#                                                      n_genes=num_genes-60,exp_filter_threshold=None,viz=False)
#         dir_strings += [dir_string]
#         data_strings += [dataset_string]

#         #Set model and run meK-Means
        
#         #Define bounds for params
#         phys_lb = [-2.0, -1.8, -1.8 ] 
#         phys_ub = [4.2, 2.5, 2.5] 
#         samp_lb = tech_tup 
#         samp_ub = tech_tup  
#         gridsize = [1,1] #Already have tech params
    

#         epochs = 10

#         # ---------------- meK-Means Inference ---------------- 
#         #Define model with bursty transcription and Poisson molecule capture/sampling
#         fitmodel = monod.cme_toolbox.CMEModel('Bursty','Poisson')

#         #Set up mminference parameters
#         inference_parameters = monod.mminference.InferenceParameters(phys_lb,phys_ub,samp_lb,samp_ub,gridsize,\
#                     dataset_string[0],fitmodel,k=clus,epochs=epochs,use_lengths = True,
#                     gradient_params = {'max_iterations':5,'init_pattern':'moments','num_restarts':1})

#         #Read in loom file with filtered barcodes
#         search_data = monod.extract_data.extract_data(d, trans, data_name,
#                     dataset_string[0], dir_string, viz=False, dataset_attr_names=attribute_names)

#         #Run inference(fit_all_grid_points()) and Save result file strings
#         full_result_string = inference_parameters.fit_all_grid_points(40,search_data) 

#         result_strings.append(full_result_string)
        

#         # ----------------  Save output ---------------- 
#         #Read in results and get cluster assignments
#         sr = [monod.analysis.load_search_results(i) for i in full_result_string]
        
        
#         cat_assigns = np.array([None]*num_cells) #cat_assigns is empty list of length = no. cell barcodes
#         for i in range(len(sr)):
#             r = sr[i]
#             cat_assigns[r.filt] = r.assigns #denotes which cluster (k), cells were assigned to

#         print('No. clus: ',len(np.unique(cat_assigns)))
        

#         meths += ['meK-Means']
#         result += [cat_assigns]
#         mats += ['U,S']
#         hyper += [clus]
#         hvgs += [num_genes]
#         data += [data_name]
#         ari += [adjusted_rand_score(true_labs,cat_assigns)]
#         ami += [adjusted_mutual_info_score(true_labs,cat_assigns)]


# meK_res['Method'] = meths
# meK_res['Clustering'] = result
# meK_res['Matrices'] = mats
# meK_res['Hyperparam'] = hyper
# meK_res['HVGs'] = hvgs
# meK_res['Data'] = data
# meK_res['ARI'] = ari
# meK_res['AMI'] = ami
    

allen_b08 with 299 HVGs


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [01:23<00:00, 41.88s/it]


mstep self.weights:  [0.55281326 0.44718674]
Q Function:  -2007059.0553544827



100%|██████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [01:10<00:00, 35.10s/it]


mstep self.weights:  [0.55623791 0.44376209]
Q Function:  -1981028.003748943



...

Process ForkPoolWorker-14463:
Traceback (most recent call last):
  File "/home/tchari/miniconda3/envs/py37/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/home/tchari/miniconda3/envs/py37/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/home/tchari/miniconda3/envs/py37/lib/python3.7/multiprocessing/pool.py", line 110, in worker
    task = get()
  File "/home/tchari/miniconda3/envs/py37/lib/python3.7/multiprocessing/queues.py", line 351, in get
    with self._rlock:
  File "/home/tchari/miniconda3/envs/py37/lib/python3.7/multiprocessing/synchronize.py", line 95, in __enter__
    return self._semlock.__enter__()
KeyboardInterrupt
Process ForkPoolWorker-14457:
Traceback (most recent call last):
  File "/home/tchari/miniconda3/envs/py37/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/home/tchari/miniconda3/envs/py37/lib/python3.7/multiproces

Process ForkPoolWorker-14448:
Traceback (most recent call last):
  File "/home/tchari/miniconda3/envs/py37/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/home/tchari/miniconda3/envs/py37/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/home/tchari/miniconda3/envs/py37/lib/python3.7/multiprocessing/pool.py", line 110, in worker
    task = get()
  File "/home/tchari/miniconda3/envs/py37/lib/python3.7/multiprocessing/queues.py", line 351, in get
    with self._rlock:
  File "/home/tchari/miniconda3/envs/py37/lib/python3.7/multiprocessing/synchronize.py", line 95, in __enter__
    return self._semlock.__enter__()
KeyboardInterrupt
Process ForkPoolWorker-14454:
Traceback (most recent call last):
  File "/home/tchari/miniconda3/envs/py37/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/home/tchari/miniconda3/envs/py37/lib/python3.7/multiproces

Process ForkPoolWorker-14465:
Traceback (most recent call last):
  File "/home/tchari/miniconda3/envs/py37/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/home/tchari/miniconda3/envs/py37/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/home/tchari/miniconda3/envs/py37/lib/python3.7/multiprocessing/pool.py", line 110, in worker
    task = get()
  File "/home/tchari/miniconda3/envs/py37/lib/python3.7/multiprocessing/queues.py", line 351, in get
    with self._rlock:
  File "/home/tchari/miniconda3/envs/py37/lib/python3.7/multiprocessing/synchronize.py", line 95, in __enter__
    return self._semlock.__enter__()
KeyboardInterrupt
Process ForkPoolWorker-14464:
Traceback (most recent call last):
  File "/home/tchari/miniconda3/envs/py37/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/home/tchari/miniconda3/envs/py37/lib/python3.7/multiproces

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

Traceback (most recent call last):
  File "/home/tchari/miniconda3/envs/py37/lib/python3.7/multiprocessing/pool.py", line 733, in next
    item = self._items.popleft()
IndexError: pop from an empty deque

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/tchari/monod/src/monod/mminference.py", line 2530, in parallelize
    x = list(tqdm(pool.imap(function, iterable), total=num_entries))  # hacky
  File "/home/tchari/miniconda3/envs/py37/lib/python3.7/site-packages/tqdm/std.py", line 1180, in __iter__
    for obj in iterable:
  File "/home/tchari/miniconda3/envs/py37/lib/python3.7/multiprocessing/pool.py", line 737, in next
    self._cond.wait(timeout)
  File "/home/tchari/miniconda3/envs/py37/lib/python3.7/threading.py", line 296, in wait
    waiter.acquire()
KeyboardInterrupt

During handling of the above exceptio

TypeError: object of type 'NoneType' has no len()

In [ ]:
#Save method, clustering, mat_in (U,S, etc), hyper (K or res), hvgs, data, ARI, NMI
meK_res.to_csv('./fits/meKruns_0209/meK_bench_results.csv',index=None)
meK_res.head()


In [96]:
# dataset_string

['./fits/meKruns_0209/gg_240209_025_allen_b08_K2_1/allen_b08']

In [37]:
meK_res.to_pickle('./fits/meKruns_0209/meK_res.pkl')
#unpickled_df = pd.read_pickle("./meK_res.pkl")

In [57]:
unpickled_df = pd.read_pickle('./fits/meKruns_0209/meK_res.pkl')
unpickled_df[unpickled_df.Data.isin(['allen_b08'])]

,Method,Clustering,Matrices,Hyperparam,HVGs,Data,ARI,AMI
0,meK-Means,"[8, 0, 9, 8, 0, 8, 0, 6, 3, 8, 0, 0, 8, 0, 3, ...","U,S",10,299,allen_b08,0.578029,0.708480
1,meK-Means,"[13, 8, 3, 13, 11, 13, 11, 14, 8, 13, 11, 12, ...","U,S",15,299,allen_b08,0.653899,0.753423
8,meK-Means,"[7, 0, 4, 1, 0, 3, 0, 5, 8, 4, 0, 9, 7, 0, 8, ...","U,S",10,1999,allen_b08,0.430570,0.688994
9,meK-Means,"[12, 12, 0, 4, 1, 12, 1, 13, 6, 0, 1, 9, 4, 12...","U,S",15,1999,allen_b08,0.389990,0.644156
12,meK-Means,"[5, 6, 9, 5, 6, 3, 6, 7, 1, 6, 6, 0, 5, 6, 1, ...","U,S",10,3999,allen_b08,0.384243,0.623599
13,meK-Means,"[4, 0, 8, 4, 0, 1, 0, 10, 2, 8, 0, 12, 4, 0, 2...","U,S",15,3999,allen_b08,0.344339,0.580940
16,meK-Means,"[1, 2, 4, 1, 2, 9, 2, 9, 2, 9, 0, 6, 1, 2, 2, ...","U,S",10,1000,allen_b08,0.449858,0.691941
17,meK-Means,"[7, 5, 8, 7, 10, 4, 10, 9, 10, 4, 10, 10, 7, 5...","U,S",15,1000,allen_b08,0.453119,0.655707


### **Leiden & K-Means**


In [26]:
#Perform standard normalization (read-depth + log1p)
def norm_mat(mat,new_tot=1e4):
    '''
    mat: cell x gene matrix
    '''  
    tots = np.sum(mat,axis=1)
    divids = (new_tot/tots)[:,None]
    mat_norm = mat*divids
    mat_norm = np.log1p(mat_norm)
    
    mat_norm[np.isnan(mat_norm)] = 0
    
    return mat_norm

In [27]:
def get_graph(mat,neigh=30):
    A = kneighbors_graph(mat, neigh, mode='connectivity', include_self=True)
    sources, targets = A.nonzero()
    weights = A[sources, targets]
    if isinstance(weights, np.matrix):
        weights = weights.A1
    g = ig.Graph(directed=False) #True in scanpy, which is odd
    g.add_vertices(A.shape[0])  # this adds adjacency.shape[0] vertices
    g.add_edges(list(zip(sources, targets)))

    g.es['weight'] = weights
    
    return g

In [28]:
# ! rm -r /home/tchari/perturbCME/notebooks/fits/LeidKMeansruns_0209

In [29]:
#Make the U,S, U+S, Concat matrices, res 0.75,1,1.5,2

!mkdir /home/tchari/perturbCME/notebooks/fits/LeidKMeansruns_0209


#Save method, clustering, mat_in (U,S, etc), hyper (K or res), hvgs, data, ARI, AMI
LK_res = pd.DataFrame()
meths = []
result = []
mats = []
hyper = []
hvgs = []
data = []
ari = []
ami = []

for d in d_strings:
    
    #Get tech param by keyword (datas) in loom paths
    ind = [i in d for i in datas]
    
    k = list(samps.Clusters[ind])[0]
    data_name = list(samps.Dataset[ind])[0]
    
    
    ds = lp.connect(d)
    num_genes = len(ds.ra[attribute_names[1]])
    num_cells = len(ds.ca[attribute_names[2]])
    U = ds.layers[attribute_names[0][0]][:,:].T
    S = ds.layers[attribute_names[0][1]][:,:].T
    true_labs = list(ds.ca['subclass_label'])
    ds.close()
    print(data_name+' with '+str(num_genes)+' HVGs')
    
    res = [0.75,1,1.5,2]
    
    #Get U,S,U_p_S,U_c_S
        
    #Norm mats
    U_norm = norm_mat(U)
    S_norm = norm_mat(S)
    U_p_S_norm = norm_mat(U+S)
    U_c_S_norm = norm_mat(np.concatenate([U,S],axis=1))
    
    g_U = get_graph(U_norm)
    g_S = get_graph(S_norm)
    g_U_p_S = get_graph(U_p_S_norm)
    g_U_c_S = get_graph(U_c_S_norm)
    
    # -------- Fit Leiden -------
    for r in res:
        
        partition_type = la.RBConfigurationVertexPartition
        #Make Knn graphs
        for g in [g_U,g_S,g_U_p_S,g_U_c_S]:
            part = la.find_partition(g, partition_type, 
                                        weights=np.array(g.es['weight']).astype(np.float64), 
                                        n_iterations=-1, resolution_parameter=r)

            labels = np.array(part.membership)
            
            # ----- Save Results ---- ARI,AMI etc
            meths += ['Leiden']
            result += [labels]
            hyper += [r]
            hvgs += [num_genes]
            data += [data_name]
            ari += [adjusted_rand_score(true_labs,labels)]
            ami += [adjusted_mutual_info_score(true_labs,labels)]
            
#             print('No. clus Leiden: ',len(np.unique(labels)))
            
        mats += ['U','S','U+S','UcatS']
        
        
        
    # -------- Fit K-Means with k -------
    
    for x in [U_norm,S_norm,U_p_S_norm,U_c_S_norm]:
        kmeans = KMeans(n_clusters=k, random_state=0).fit(x)
        labs = kmeans.labels_
        
        meths += ['K-Means']
        result += [labs]
        hyper += [k]
        hvgs += [num_genes]
        data += [data_name]
        ari += [adjusted_rand_score(true_labs,labs)]
        ami += [adjusted_mutual_info_score(true_labs,labs)]
        
#         print('No. clus K-Means: ',len(np.unique(labs)))
        
    mats += ['U','S','U+S','UcatS']

    
LK_res['Method'] = meths
LK_res['Clustering'] = result
LK_res['Matrices'] = mats
LK_res['Hyperparam'] = hyper
LK_res['HVGs'] = hvgs
LK_res['Data'] = data
LK_res['ARI'] = ari
LK_res['AMI'] = ami

mkdir: cannot create directory ‘/home/tchari/perturbCME/notebooks/fits/LeidKMeansruns_0209’: File exists
allen_b08 with 299 HVGs
cl5 with 2000 HVGs


/home/tchari/.local/lib/python3.7/site-packages/ipykernel_launcher.py:7: RuntimeWarning: divide by zero encountered in true_divide
  import sys
/home/tchari/.local/lib/python3.7/site-packages/ipykernel_launcher.py:8: RuntimeWarning: invalid value encountered in multiply
  


allen_b02h01 with 299 HVGs


/home/tchari/.local/lib/python3.7/site-packages/ipykernel_launcher.py:7: RuntimeWarning: divide by zero encountered in true_divide
  import sys
/home/tchari/.local/lib/python3.7/site-packages/ipykernel_launcher.py:8: RuntimeWarning: invalid value encountered in multiply
  


cl5 with 300 HVGs


...

/home/tchari/.local/lib/python3.7/site-packages/ipykernel_launcher.py:7: RuntimeWarning: divide by zero encountered in true_divide
  import sys
/home/tchari/.local/lib/python3.7/site-packages/ipykernel_launcher.py:8: RuntimeWarning: invalid value encountered in multiply
  


allen_b02h01 with 4000 HVGs
cl3 with 4000 HVGs


/home/tchari/.local/lib/python3.7/site-packages/ipykernel_launcher.py:7: RuntimeWarning: divide by zero encountered in true_divide
  import sys
/home/tchari/.local/lib/python3.7/site-packages/ipykernel_launcher.py:8: RuntimeWarning: invalid value encountered in multiply
  


allen_b02h01 with 2000 HVGs
cl3 with 299 HVGs


/home/tchari/.local/lib/python3.7/site-packages/ipykernel_launcher.py:7: RuntimeWarning: divide by zero encountered in true_divide
  import sys
/home/tchari/.local/lib/python3.7/site-packages/ipykernel_launcher.py:8: RuntimeWarning: invalid value encountered in multiply
  


In [ ]:
#Save method, clustering, mat_in (U,S, etc), hyper (K or res), hvgs, data, ARI, NMI
# LK_res.to_csv('./fits/LeidKMeansruns_0209/LeidKMeans_bench_results.csv',index=None)
# LK_res.head()



In [30]:
LK_res.to_pickle('./fits/LeidKMeansruns_0209/LK_res.pkl')
#unpickled_df = pd.read_pickle("./meK_res.pkl")

In [45]:
unpickled_df_LK = pd.read_pickle("./fits/LeidKMeansruns_0209/LK_res.pkl")
unpickled_df_LK

,Method,Clustering,Matrices,Hyperparam,HVGs,Data,ARI,AMI
0,Leiden,"[1, 3, 4, 1, 0, 7, 2, 8, 3, 1, 0, 2, 1, 3, 3, ...",U,0.75,299,allen_b08,0.695484,0.829751
1,Leiden,"[1, 2, 6, 1, 0, 8, 3, 9, 2, 10, 0, 3, 1, 2, 2,...",S,0.75,299,allen_b08,0.675494,0.845487
2,Leiden,"[1, 2, 6, 1, 0, 8, 3, 9, 2, 10, 0, 3, 1, 2, 2,...",U+S,0.75,299,allen_b08,0.669919,0.843539
3,Leiden,"[4, 1, 7, 4, 0, 9, 3, 10, 1, 2, 0, 3, 2, 1, 1,...",UcatS,0.75,299,allen_b08,0.576599,0.803110
4,Leiden,"[4, 2, 6, 4, 0, 8, 1, 9, 2, 3, 0, 1, 3, 2, 2, ...",U,1.00,299,allen_b08,0.620979,0.806199
...,...,...,...,...,...,...,...,...
315,Leiden,"[5, 5, 5, 5, 5, 2, 5, 5, 5, 5, 2, 5, 5, 5, 2, ...",UcatS,2.00,299,cl3,0.095006,0.204309
316,K-Means,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",U,3.00,299,cl3,0.030605,0.032775
317,K-Means,"[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ...",S,3.00,299,cl3,0.035307,0.037991
318,K-Means,"[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ...",U+S,3.00,299,cl3,0.034961,0.038963


In [58]:
unpickled_df_LK[unpickled_df_LK.Method.isin(['K-Means']) & unpickled_df_LK.Data.isin(['allen_b08'])]

,Method,Clustering,Matrices,Hyperparam,HVGs,Data,ARI,AMI
16,K-Means,"[1, 3, 7, 1, 2, 5, 6, 4, 3, 9, 2, 6, 1, 3, 3, ...",U,10.0,299,allen_b08,0.620592,0.807500
17,K-Means,"[0, 8, 9, 0, 2, 7, 3, 4, 8, 0, 2, 3, 0, 8, 8, ...",S,10.0,299,allen_b08,0.659600,0.830449
18,K-Means,"[0, 6, 4, 0, 5, 8, 1, 7, 6, 2, 5, 1, 2, 6, 6, ...",U+S,10.0,299,allen_b08,0.626460,0.822118
19,K-Means,"[0, 8, 5, 0, 6, 4, 2, 7, 8, 0, 6, 2, 0, 8, 8, ...",UcatS,10.0,299,allen_b08,0.658582,0.831881
96,K-Means,"[6, 5, 7, 6, 4, 2, 3, 9, 5, 6, 4, 3, 6, 5, 5, ...",U,10.0,1999,allen_b08,0.660154,0.830761
97,K-Means,"[7, 4, 5, 7, 0, 9, 6, 8, 4, 1, 0, 6, 1, 4, 4, ...",S,10.0,1999,allen_b08,0.640628,0.842625
98,K-Means,"[1, 6, 4, 1, 0, 3, 9, 2, 6, 1, 0, 9, 1, 6, 6, ...",U+S,10.0,1999,allen_b08,0.667883,0.847009
99,K-Means,"[2, 0, 7, 2, 4, 6, 1, 9, 0, 2, 4, 1, 2, 0, 0, ...",UcatS,10.0,1999,allen_b08,0.671042,0.849206
136,K-Means,"[4, 7, 9, 4, 2, 6, 1, 3, 7, 0, 2, 1, 0, 7, 7, ...",U,10.0,3999,allen_b08,0.634984,0.825675
137,K-Means,"[1, 0, 6, 1, 2, 8, 9, 5, 0, 8, 2, 9, 1, 0, 0, ...",S,10.0,3999,allen_b08,0.566372,0.803837
